**Table of Contest**

* [Can Computer Vision spot Distracted Drivers?](#chapter1)
* [Importing Libraries](#chapter2)
* [Loading Data](#chapter3)
* [Defining Model Architecture](#chapter4)
* [Training Model](#chapter5)
* [Modifying Pretrained Resnet34 Model](#chapter6)
* [Comparison of Models](#chapter7)

<h1 style="color:Red;">Can Computer Vision Spot Distracted Drivers?</h1> <a class="anchor"  id="chapter1"></a>

<h3 style="color:Red;">Identify the Problem:</h3>
      What is the goal?  ->
          The main goal is to detect the distracted drivers.<br>
      Why do we need that?  -> 
          A scenario would be the safe driving.<br>
          Six percent of all drivers involved in fatal crashes in 2019 were reported as distracted at the time of the crashes. 
          Nine percent of drivers 15 to 20 years old  involved in fatal crashes were reported as distracted.
          
   [Full Report here](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/813111).
 <br>

<h3 style="color:Red;">Success Criteria:</h3>
    Measure the Effectiveness of the solution.<br>
    We will use accuracy as a success criteria.<br><br>

<h3 style="color:Red;">Technique Used:</h3><br>
    More precisely we want to clasify the causes of distraction.<br>
    We have 10 classes to predict the causes of distraction.<br>
        c0: normal driving<br>
        c1: texting - right<br>
        c2: talking on the phone - right<br>
        c3: texting - left<br>
        c4: talking on the phone - left<br>
        c5: operating the radio<br>
        c6: drinking<br>
        c7: reaching behind<br>
        c8: hair and makeup<br>
        c9: talking to passenger<br>

<h1 style="color:Red;">Importing Libraries</h1><a class="anchor"  id="chapter2"></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import torchvision
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torch.utils.data import random_split
from torch.utils.data import Dataset,DataLoader
from torchvision.utils import make_grid
import torch.nn.functional as F
from torchvision import models
from tqdm.notebook import tqdm
%matplotlib inline

In [ ]:
# Train path
train_path = "../input/state-farm-distracted-driver-detection/imgs/train"
# Test path
test_path = "../input/state-farm-distracted-driver-detection/imgs/test"

train_length = 0
for clss in os.listdir(train_path):
    print("%s size: %d" % (clss, len(os.listdir(os.path.join(train_path, clss)))))
    train_length += len(os.listdir(os.path.join(train_path, clss)))
print("Train size: %d" % train_length)
print("Test Size: %d" % len(os.listdir(test_path)))

The training data is equally distributed.

<h1 style="color:Red;">Loading Data</h1><a class="anchor"  id="chapter3"></a>

    The Train Dataset has over 22.5k images.
    The Test Dataset has over 80k images without labels.
    We will use ImageFolder to load train dataset.

In [ ]:
# Data Transforms and Augmentation
train_transforms = T.Compose([ T.Resize((64, 64)),
                               T.RandomAdjustSharpness(2),
                               T.RandomRotation((-15, 15)),
                               T.ColorJitter(brightness=.5, hue=.3),
                               T.ToTensor(),
                             ])

# Loading Data using ImageFolder
train_ds = ImageFolder(train_path, train_transforms)
classes = train_ds.classes
print(classes)

# Splitting into train-val set
val_pct = .1
val_size = int(val_pct * len(train_ds))
train_ds ,valid_ds = random_split(train_ds, [len(train_ds)-val_size, val_size])

# Data Loader
batch_size = 64
train_dl = DataLoader(train_ds, batch_size, shuffle = True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=2, pin_memory=True)

**Examples from train dataset**

In [ ]:
def show_batch(dl):
  for images,labels in dl:
    fig, ax = plt.subplots(figsize=(8, 16))
    ax.set_xticks([]);ax.set_yticks([])
    ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0).clamp(0, 1))
    break

In [ ]:
show_batch(train_dl)

<h1 style="color:Red;">Defining Model Architecture</h1><a class="anchor"  id="chapter4"></a>
    
    We will define a ConvBlock and use that ConvBlock 4 times.
    The ConvBlocks are;
    >Input(image) size is [64, 3, 64, 64].
    > ConvBlock(in_channels=3, out_channels=64)(image) >>> output size is [64, 64, 32, 32].
    > ConvBlock(64, 128)(out) >>> [64, 128, 16, 16].
    > ConvBlock(128, 256)(out) >>> [64, 256, 8, 8].
    > ConvBlock(256, 512)(out) >>> [64, 512, 4, 4].
    
    
    After ConvBlocks we have fully connected layer.
    > Flatten                     >>> [64, 512*4*4] = [64, 8192]
    > Linear(512*4*4, 500)        >>> [64, 500]
    > Linear(500, 10)             >>> [64, 10]

In [ ]:
def ConvBlock(in_channels, out_channels):
    """
        >>>conv_block = ConvBlock(3, 64)
        >>>noise = torch.rand((32, 3, 64, 64))
        >>>out = conv_block(noise)
        >>>out.size()
        
        >>>torch.Size([1, 64, 32, 32])
    """
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=2, padding="same"),
              nn.ReLU(),
              nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True),
              nn.MaxPool2d(2),
    ]
    return nn.Sequential(*layers)

class DistractedDriverDetectionModel(nn.Module):
    def __init__(self, input_size=3, num_classes=10):
        super(DistractedDriverDetectionModel, self).__init__()
        self.model = nn.Sequential(
            ConvBlock(in_channels=input_size, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            ConvBlock(in_channels=256, out_channels=512),
        )

        self.fully_connected_layer = nn.Sequential(
            nn.Dropout(0.5),
            nn.Flatten(),
            nn.Linear(512*4*4, 500),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(500, num_classes),
            nn.Softmax(1),
        )
    
    def forward(self, x):
        output = self.model(x)
        output = self.fully_connected_layer(output)
        return output

<h1 style="color:Red;">Training the Model</h1><a class="anchor"  id="chapter5"></a>

    First we will set our DistractionDetection Model and Parameters.
    After that we will train the model about 30 epochs and we'll see the results.

In [ ]:
# Device configuration
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(device)

# Model configuration
model = DistractedDriverDetectionModel(input_size=3, num_classes=10)

model = model.to(device)

In [ ]:
# Set Parameters
epochs = 30
lr = 1e-4
opt_func = torch.optim.RMSprop

In [ ]:
def accuracy(outputs,labels):
  _,preds = torch.max(outputs,dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))
        
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = []
    for batch in val_loader:
        images,labels = batch
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)                     
        loss = F.cross_entropy(out, labels)     
        acc = accuracy(out, labels)             
        outputs.append({"val_loss":loss.detach(), "val_acc":acc})
        
    batch_losses = [x["val_loss"] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()     # Combine Losses
    batch_accs = [x["val_acc"] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()        # Combine Accuracies
    return {"val_loss":epoch_loss.item(),"val_acc":epoch_acc.item()}

torch.cuda.empty_cache()
history = []

#Set up custom optimizer with weight decay
optimizer = opt_func(model.parameters(), lr)

for epoch in range(epochs):
    # Training Phase
    model.train()
    train_losses = []
    for batch in tqdm(train_dl):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(out,labels)
        train_losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
    
    # Validation Phase
    result = evaluate(model, valid_dl)
    result["train_loss"] = torch.stack(train_losses).mean().item()
    print("Epoch [{}/{}], train_loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.4f}".format(epoch, epochs,
                                                                                         result["train_loss"],
                                                                                         result["val_loss"],
                                                                                         result["val_acc"]))
    history.append(result)

<h2 style="color:Red;">Modifying Pretrained Resnet34 Model</h2><a class="anchor"  id="chapter6"></a>

    We will modify the pretrained ResNet34 model for our case.
    Then we will train the network about 8 epochs.

In [ ]:
class ResNet34(nn.Module):
    def __init__(self,num_classes,pretrained=True):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=pretrained)
        # Replace last layer
        self.network.fc = nn.Linear(self.network.fc.in_features, num_classes)

    def forward(self,x):
        return self.network(x)

In [ ]:
# Device configuration
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Model configuration
pretrained_model = ResNet34(num_classes=10)

pretrained_model.to(device)

In [ ]:
# Setting parameters
epochs = 8
lr = 1e-4
opt_func = torch.optim.Adam

In [ ]:
"""
    Training step
"""
#Set up custom optimizer with weight decay
optimizer = opt_func(pretrained_model.parameters(), lr)

# Saving results for the evaluation of the model and comperison with other model.
history2 = []

for epoch in range(epochs):
    # Training Phase
    pretrained_model.train()
    train_losses = []
    lrs = []
    for batch in tqdm(train_dl):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        out = pretrained_model(images)                    
        loss = F.cross_entropy(out, labels)
        train_losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # Validation Phase
    result2 = evaluate(pretrained_model,valid_dl)
    result2["train_loss"] = torch.stack(train_losses).mean().item()
    # result2["lrs"] = lrs  # I won't add learning rate changes to the history
    print("Epoch [{}/{}], train_loss : {:.4f}, val_loss : {:.4f}], val_acc : {:.4f}".format(epoch, epochs,
                                                                                         result2["train_loss"],
                                                                                         result2["val_loss"],
                                                                                         result2["val_acc"]))
    history2.append(result2)

<h1 style="color:Red;">Comparison of Models</h1><a class="anchor"  id="chapter7"></a>
    
    The Models got  almost same accuracy.
    Simple Architecture got ~%98.40. The best accuracy was ~%98.69 at 26th epoch.
    Pretrained ResNet34 Architecture got ~%98.78. The best accuracy was ~%99.09 at 7th epoch.
    We saved these as val_acc and train_loss in history(simple architecture) and history2(pretrained ResNet34 architecture).
    There are subtle differences between loss changes in simple architecture. If we plot these losses the line would look like straight line. 

In [ ]:
plt.plot([*range(1, 9)], [x["val_acc"]*100 for x in history2], c="green")
plt.plot(7, history2[6]["val_acc"]*100, marker="o", color="red")
plt.title("Accuracy of the pretrained ResNet34 model \n The Best acc at 7th epoch")
plt.text(7.1, 99.1, "{}".format(history2[6]["val_acc"]*100), c="red");

In [ ]:
plt.plot([*range(1, 31)], [x["val_acc"]*100 for x in history], c="green")
plt.plot(26, history[25]["val_acc"]*100, marker="o", color="red")
plt.title("Accuracy of the simple model \n The Best acc at 26th epoch")
plt.text(26.1, history[25]["val_acc"]*100 + 0.2, "{:.2f}".format(history[25]["val_acc"]*100), c="red");